## Read The Datasets

## Overview of Each Dataset


### Dataset High Level Overview

For each Dataset get the ff. (tabular)
- Number of Rows
- Number of Columns
- Number of Null Values
- Number Columns with Null Values
- List of Columns with Null

### Dataset Detailed Info

Create a tabular data of the ff.

- Column name
- Null amount
- Null percentage among the respective dataset
- Data type
- total categorical entries

## Exploratory Data Analysis (EDA)

### How many orders we have for each status?

### What is the total number of orders per month, per week, per time of the day (Dawn, Morning, Afternoon, Night)?

Line Chart

1. Is there any growing trend on brazilian e-commerce?
2. On what day of week brazilians customers tend to do online purchasing?
3. What time brazilians customers tend do buy (Dawn, Morning, Afternoon or Night)?

#### Transform the order_purchase_timestamp to get the following:

Date Dimension:

- Year Month
- Day of Week
- Time of Day

### Create a Comparison Between 2017 and 2018 Total Orders (Month by Month and YoY)

### Geospatial Visualization

#### Data Processing Required
1. Merge the orders data to order_items data;
2. Use an API (brazilian government) to return the region of each customer_state;
3. Purpose useful charts to answear business questions.

#### Total Number of Customer Order by Region, State, City

- Comparison of Each Region Orders per Month
- Top 10 Brazilian Cities with More Orders
- Total of Customers Orders by State

#### How customers are distributed in Brazil? (Map and Heatmap)

#### Total Orders vs. Total Amount Sold
- Per Month (Line)
- Per year (Number)

#### How the total sales (sum of price) are concentraded in brazilian states?

- Average price per Customer State
- Total price by Customer State

#### What are the best states to buy in Brazil? An analysis on sales, freight and delivery time

- Average Freight Paid for Online Shopping
- Top 5 States with Highest Avg Freight Value
- Top 5 States with Lowest Average Freight Value

- Average Delay (days) for delivery for online shopping
- Top 5 States with Highest Average Time to Delivery
- Top 5 States with Lowest Average Time to Delivery
- Average Difference between delivery and estimated date
- Top 5 States Delivery is Really Fast
- Top 5 States Delivery is not so Fast


## Payment Type Analysis

- Total Transactions by Payment Type
- Distribution of Payment Installments
- Total Orders Purcharsed By Month